In [4]:
! pip3 install torch torchvision

  Using cached https://files.pythonhosted.org/packages/7e/60/66415660aa46b23b5e1b72bc762e816736ce8d7260213e22365af51e8f9c/torch-1.0.0-cp36-cp36m-manylinux1_x86_64.whl
tcmalloc: large alloc 1073750016 bytes == 0x60a14000 @  0x7fa8979842a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641
  Using cached https://files.pythonhosted.org/packages/ca/0d/f00b2885711e08bd71242ebe7b96561e6f6d01fdb4b9dcf4d37e2e13c5e1/torchvision-0.2.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/85/5e/e91792f198bbc5a0d7d3055ad552bc4062942d27eaf75c3e2783cf64eae5/Pillow-5.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0
  [PIL]
You must restart the runtime in orde

In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

#사이킷런
from sklearn.datasets import load_digits
from sklearn import datasets, model_selection

#판다스
import pandas as pd

#맷플랏
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline

import numpy as np

In [10]:
#cuda 사용가능한지 확인
torch.cuda.is_available()

True

## 데이터 로드

In [0]:
train_df = pd.read_csv('sample_data/mnist_train_small.csv')
test_df = pd.read_csv('sample_data/mnist_test.csv')

In [11]:
train_df.shape

(19999, 785)

In [12]:
train_df.head()

,6,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.581,0.582,0.583,0.584,0.585,0.586,0.587,0.588,0.589,0.590
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
test_df.head()

,7,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## data set 나누고 텐서생성

In [0]:
train_Y = train_df["6"]
train_X = train_df.drop("6", axis=1)

test_Y = test_df["7"]
test_X = test_df.drop("7", axis=1)

In [0]:
#X 데이터는 convolution을 위해 28*28 모양으로 바꾼다

train_X = torch.from_numpy(train_X.as_matrix()).float()
train_X = train_X.reshape((len(train_X),1,28,28))

train_Y = torch.from_numpy(train_Y.as_matrix()).long()

test_X = torch.from_numpy(test_X.as_matrix()).float()
test_X = test_X.reshape((len(test_X),1,28,28))

test_Y = torch.from_numpy(test_Y.as_matrix()).long()

In [0]:
train_X = train_X.cuda()
train_Y = train_Y.cuda()

test_X = test_X.cuda()
test_Y = test_Y.cuda()

In [33]:
print(train_X.shape)
print(train_Y.shape)
print(test_X.shape)
print(test_Y.shape)

torch.Size([19999, 1, 28, 28])
torch.Size([19999])
torch.Size([9999, 1, 28, 28])
torch.Size([9999])


In [0]:
train = TensorDataset( train_X, train_Y)

In [0]:
#미니 배치로 분활
train_loader = DataLoader(train, batch_size=100, shuffle=True)

## 모델

In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    #합성곱
    self.conv1 = nn.Conv2d(1,6,5) 
    self.conv2 = nn.Conv2d(6,16,5)
    #전결합층
    self.fc1 = nn.Linear(256,64)
    self.fc2 = nn.Linear(64, 10)
    
    
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)),2)
    x = F.max_pool2d(F.relu(self.conv2(x)),2)
    x = x.view(-1,256)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return F.log_softmax(x)  

In [37]:
model = Net()
model.cuda()

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

In [0]:
#오차함수
loss_fuction = nn.CrossEntropyLoss()

In [0]:
#최적화
optimizer = optim.Adam(model.parameters(),lr=0.001)

## 학습

In [42]:
# 학습 시작
for epoch in range(100):
  total_loss = 0
  
  #분할해둔 데이터를 꺼내옴
  for train_x, train_y in train_loader:
    #계산 그래프 구성
    train_x, train_y = Variable(train_x), Variable(train_y)
    #경사 초기화
    optimizer.zero_grad()
    #순전파 계산
    output = model(train_x)
    #오차 계산 
    loss = loss_fuction(output, train_y)
    #역전파 계산
    loss.backward()
    #가중치 업데이트
    optimizer.step()
    #누적 오차 계산
    total_loss += loss.item()
  
  #100회 반복마다 누적 오차 출력
  if (epoch + 1) % 1 == 0:
    print(epoch+1, total_loss)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


1 30.84396526031196
2 21.824635973200202
3 18.9081520694308
4 17.166868158150464
5 15.949102499522269
6 15.493601221824065
7 14.346773315686733
8 13.930119183380157
9 13.128149687312543
10 12.326443547848612
11 12.012490916531533
12 11.486043064622208
13 10.889871281338856
14 10.34524796367623
15 9.938549081562087
16 9.284221447072923
17 9.109948897734284
18 8.554813067778014
19 8.444832838140428
20 7.9354002113686875
21 7.594571825757157
22 7.5775023099267855
23 7.150081589934416
24 6.641065237228759
25 6.554185159446206
26 6.2569583630538546
27 6.364699893340003
28 5.886697267007548
29 5.687382079602685
30 5.761219875159441
31 5.923940954264253
32 5.892003140528686
33 5.044420484773582
34 4.833400839386741
35 4.489592458441621
36 4.401285812447895
37 4.385967077250825
38 4.542104126157938
39 4.3504335965408245
40 4.685985304797214
41 4.727992559870472
42 3.9936338202969637
43 3.570688269996026
44 3.550776778400177
45 4.045297735989152
46 4.296167558444722
47 3.922100575415243
48 5.20

In [0]:
test_x, test_y = Variable(test_X), Variable(test_Y)

In [44]:
result = torch.max(model(test_x).data, 1)[1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [47]:
test_y_np = test_y.data.cpu().numpy()
result_np = result.cpu().numpy()
acc = sum(test_y_np == result_np ) / len(test_y_np)
acc

0.9628962896289629